In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

# Static Configurations

> In this module, we configure stuff.

In [ ]:
#| default_exp configuration

In [ ]:
#| export

from pathlib import Path
import thucy

In [ ]:
#| exports

class Config:
    """Global configuration for the thucy project"""
    
    def __init__(self):
        # Path Configs
        self.project_root = Path(thucy.__file__).parent.parent
        self.expertiments_dir = self.project_root / "experiments"
        self.results_dir = self.expertiments_dir / "results"
        
        # Agentic Configs
        self.experts_model = "gpt-5-mini"
        self.lead_model = "gpt-5"
        
        # Max turns
        self.sql_expert_max_turns = 30
        self.schema_expert_max_turns = 30
        self.data_expert_max_turns = 30
        
        # MCP ToolBox
        self.genai_server_url = "http://127.0.0.1:5000"

In [ ]:
#| exports

config = Config()

In [ ]:
config.project_root

PosixPath('/Users/michaeltheologitis/Code/thucy')

In [ ]:
#| export
from dotenv import set_key

In [ ]:
#| export

def set_project_env_variable(name: str, key: str):
    """Sets an environment variable for the project and saves it to the project .env file."""
    env_path = config.project_root / ".env"
    
    # Create file if it doesn't exist
    if not env_path.exists():
        env_path.touch()
    
    # This handles overwriting automatically
    set_key(env_path, name, key)

In [ ]:
set_project_env_variable("OPENAI_API_KEY", "sk-xxsaxxxxx")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()